In [147]:
import pandas as pd
from astropy.io import fits
from glob import glob
import numpy as np
import os
import astropy.units as u
from tqdm import tqdm
import ads
from astroquery.simbad import Simbad

In [148]:
Simbad.add_votable_fields('ids')

In [149]:
# dirs = glob('/Volumes/Nibelheim/hst/data/*')
# df = pd.DataFrame(columns=['propid', 'pi_l', 'pi_f', 'date', 'ra', 'dec', 'name', 'mode'])
# idx = 0
# for dir in tqdm(dirs, position=0, leave=True):
#     if os.path.isdir(dir):
#         fnames=glob(f'{dir}/*.fits')
#         for fname in fnames:
#             try:
#                 hdr = fits.getheader(fname)
#             except:
#                 continue
#             df.loc[idx] = [hdr['PROPOSID'], hdr['PR_INV_L'], hdr['PR_INV_F'], hdr['DATE-OBS'], hdr['RA_TARG'], hdr['DEC_TARG'], hdr['TARGNAME'], hdr['PHOTMODE']]
#             idx += 1
# grp = df.groupby(['name', 'propid']).max()
# grp['nfiles'] = np.asarray(df.groupby(['name', 'propid']).count()['mode'])
# grp.to_csv('literature_cat.csv')

In [150]:
grp = pd.read_csv('literature_cat.csv', index_col=[0, 1])

In [151]:
grp = grp[grp.nfiles > 20]

In [152]:
grp.sort_values('date')

,,pi_l,pi_f,date,ra,dec,mode,nfiles
name,propid,,,,,,,
G191B2B,11926,Deustua,Susana,2010-08-09,76.377550,52.831094,WFC3 IR G141,26
HAT-13,12181,Deming,Drake,2010-09-08,129.882390,47.351946,WFC3 IR G141,200
WASP-12,12181,Deming,Drake,2010-10-10,97.636667,29.672222,WFC3 IR G141,28
HD-189733,11740,Pont,Frederic,2010-11-10,300.182120,22.710114,WFC3 IR G141,216
TRES-3,12181,Deming,Drake,2011-03-02,268.029079,37.546266,WFC3 IR G141,220
...,...,...,...,...,...,...,...,...
HAT-P-70B,16307,Fu,Guangwei,2021-01-27,74.552340,9.997992,WFC3 IR G141,72
L-98-59,15856,Barclay,Thomas,2021-02-24,124.533263,-68.314902,WFC3 IR G141,754
HD-134004,16450,Lothringer,Joshua,2021-03-05,227.270309,-42.704976,WFC3 IR G141,152


In [153]:
propids = np.unique(np.asarray(list(grp.index))[:, 1])

In [164]:
grp['simbad'] = np.nan

In [165]:
%%capture --no-stderr

lit = pd.DataFrame(columns=['propid', 'name', 'simbad', 'title', 'bibcode', 'author', 'fa', 'altnames'])
jdx = 0
pref = ["TRAPPIST", '* 55 Cn', 'WASP', 'CoRoT', 'XO', 'HAT', "Kepler", 'K2', 'GJ', 'L', 'HD', 'HR', "KIC", "TOI", "HIP", "EPIC", "TIC"]

In [166]:
def run(idx):
    n = idx[0]
    if n[-1].isalpha():
        n = n[:-1]
    funcs = [Simbad.query_region, Simbad.query_object, Simbad.query_object, Simbad.query_region]
    args = [(f'{g.ra} {g.dec}', 3*u.arcsecond), (n), (n.replace('-', '', 1)), (f'{g.ra} {g.dec}', 20*u.arcsecond)]
    for sdx in range(len(funcs)):
        try:
            q = funcs[sdx](*args[sdx])
        except:
            continue
        if not isinstance(q, (dict, type(None))):
            break
    altnames = q[np.argmax([len(idx.split('|')) for idx in q['IDS']])]['IDS'].split('|')
    altnames = np.unique([m[:-1] if np.any([m.endswith(l) for l in 'bcdefghi']) else m for m in altnames])
    #name_strings = '({0})'.format(' OR '.join([f"'{name}'" for name in altnames]))


    best_pref = pref[np.sort([np.where([a.startswith(b) for b in pref])[0][0] for a in altnames if len(np.where([a.startswith(b) for b in pref])[0]) > 0])[0]]
    best_name = [a for a in altnames if a.startswith(best_pref)][0]

    #name = np.unique([m[:-1] if np.any([m.endswith(l) for l in 'bcdefghi']) else m for m in q['MAIN_ID']])[0]

    name = [best_name[:-1] if np.any([best_name.endswith(l) for l in 'bcdefghi']) else best_name][0]
    return name, altnames

In [167]:
for idx, g in tqdm(grp.iterrows()):
    name, altnames = run(idx)
    if name == '* 55 Cn':
        name = '55 Cancri'
    searches = [f"{name} hst wfc3", f"{name}b hst wfc3", f"{name}c hst wfc3", f"hst wfc3 object={name}", f"hst wfc3 {idx[1]}"]
    for search in searches:
        papers = ads.SearchQuery(q=search)
        for paper in papers:
            lit.loc[jdx] = idx[1], idx[0], name, paper.title[0].lower(), paper.bibcode, paper.author, ' '.join(np.sort(paper.first_author.replace(',', '').split(' '))), altnames
            jdx += 1
    grp.loc[idx, 'simbad'] = name
    lit = lit.drop_duplicates(['title', 'propid', 'name', 'fa'])

0it [00:00, ?it/s]WARNING: InputWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees. [astroquery.utils.commons]
1it [00:02,  2.58s/it]WARNING: InputWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees. [astroquery.utils.commons]
2it [00:04,  2.48s/it]WARNING: InputWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees. [astroquery.utils.commons]
3it [00:07,  2.51s/it]WARNING: InputWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees. [astroquery.utils.commons]
4it [00:10,  2.68s/it]WARNING: InputWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees. [astroquery.utils.commons]
/Users/ch/Library/Caches/pypoetry/virtualenvs/ombre-vf38qw1p-py3.8/lib/python3.8/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the re

In [178]:
lit.to_csv('literature_bib.csv', index=False)

In [183]:
grp = pd.read_csv('literature_cat.csv', index_col=['simbad', 'propid'])

In [62]:
lit.to_csv('literature_bib.csv', index=False)

In [182]:
bad = ['AAS', 'AUGFM', 'AGUFMP007', 'AcASn', 'DPS', "EPSC", "ESS", 'IAUGA', 'IAUS', 'cosp', 'ebi', 'epsc', 'hst', 'jwst', 'sptz', 'ycat']
articles = [np.all([(b not in code) for b in bad]) for code in lit.bibcode]
bad = ['vizier']
articles &= np.asarray([np.all([(b not in title) for b in bad]) for title in lit.title])

In [184]:
#grp = df.groupby(['name', 'propid']).max()
#grp['nfiles'] = np.asarray(df.groupby(['name', 'propid']).count()['mode'])
grp = grp.join(lit[articles].groupby(['simbad', 'propid'])['title'].apply(list))
grp = grp.join(lit[articles].groupby(['simbad', 'propid'])['bibcode'].apply(list))
grp = grp.join(lit[articles].groupby(['simbad', 'propid'])['altnames'].first())
grp = grp.join(lit[articles].groupby(['simbad', 'propid'])['simbad'].first())
#tab = grp[grp.nfiles > 20].reset_index().groupby(['simbad']).first().sort_values('date')

In [211]:
grp['jd'] = np.asarray([Time.strptime(t, '%Y-%m-%d').jd for t in np.asarray(grp['date'])])

In [222]:
grp.sort_values('jd').head(50)

,,name,pi_l,pi_f,date,ra,dec,mode,nfiles,title,bibcode,altnames,simbad,jd
simbad,propid,,,,,,,,,,,,,
LS V +52 21,11926,G191B2B,Deustua,Susana,2010-08-09,76.377550,52.831094,WFC3 IR G141,26,[ultralytics/yolov5: v5.0 - yolov5-p6 1280 mod...,"[2021zndo...3908559J, 2021arXiv210801382V, 201...","[** WNO 44B, 1RXS J050525.8+524913, 1SWASP J...",LS V +52 21,2455417.5
HAT-P-13,12181,HAT-13,Deming,Drake,2010-09-08,129.882390,47.351946,WFC3 IR G141,200,NaN,NaN,NaN,NaN,2455447.5
WASP-12,12181,WASP-12,Deming,Drake,2010-10-10,97.636667,29.672222,WFC3 IR G141,28,[exoplanet transit spectroscopy using wfc3: wa...,"[2013ApJ...779..128M, 2015ApJ...801...86K, 201...","[2MASS J06303279+2940202, Gaia DR1 34352828624...",WASP-12,2455479.5
GJ 4130,11740,HD-189733,Pont,Frederic,2010-11-10,300.182120,22.710114,WFC3 IR G141,216,NaN,NaN,NaN,NaN,2455510.5
TIC 116264089,12181,TRES-3,Deming,Drake,2011-03-02,268.029079,37.546266,WFC3 IR G141,220,NaN,NaN,NaN,NaN,2455622.5
WASP-12,12230,WASP-12,Swain,Mark,2011-04-16,97.636639,29.672278,WFC3 IR G141,1160,[exoplanet transit spectroscopy using wfc3: wa...,"[2013ApJ...779..128M, 2015ApJ...801...86K, 201...","[2MASS J06303279+2940202, Gaia DR1 34352828624...",WASP-12,2455667.5
WASP-18,12181,WASP-18,Deming,Drake,2011-06-25,24.354295,-45.677887,WFC3 IR G141,312,[estimating dayside effective temperatures of ...,"[2019MNRAS.489..941P, 2019A&A...625A.136A, 202...","[2MASS J01372503-4540404, CD-46 449, CPC 0 ...",WASP-18,2455737.5
WASP-19,12181,WASP-19,Deming,Drake,2011-07-01,148.416812,-45.659142,WFC3 IR G141,550,[exoplanet transit spectroscopy using wfc3: wa...,"[2013ApJ...779..128M, 2013MNRAS.434.3252H, 201...","[2MASS J09534008-4539330, ASTEP A-009-1934-226...",WASP-19,2455743.5
WASP-17,12181,WASP-17,Deming,Drake,2011-07-08,239.962263,-28.061785,WFC3 IR G141,132,[exoplanet transit spectroscopy using wfc3: wa...,"[2013ApJ...779..128M, 2021arXiv210813721S, 201...","[1SWASP J155950.94-280342.3, 2MASS J15595095-2...",WASP-17,2455750.5


In [239]:
grp[((Time.now().jd - grp.jd)/365) < 2]

,,name,pi_l,pi_f,date,ra,dec,mode,nfiles,title,bibcode,altnames,simbad,jd
simbad,propid,,,,,,,,,,,,,
K2-135,15333,BD-02-5958,Crossfield,Ian,2020-01-08,351.772222,-1.285074,WFC3 IR G141,523,NaN,NaN,NaN,NaN,2458856.5
LTT 9779,16457,CD-38-15670,Edwards,Billy,2021-06-12,358.669393,-37.628340,WFC3 IR G141,69,NaN,NaN,NaN,NaN,2459377.5
HAT-P-2,16194,HAT-P-2,Desert,Jean-Michel,2020-12-21,245.151412,41.047917,WFC3 IR G141,279,[detection of helium in the atmosphere of the ...,"[2018ApJ...868L..34M, 2017AJ....154...39D, 201...","[2MASS J16203635+4102531, AG+41 1379, BD+41 2...",HAT-P-2,2459204.5
HD 287325,16307,HAT-P-70B,Fu,Guangwei,2021-01-27,74.552340,9.997992,WFC3 IR G141,72,NaN,NaN,NaN,NaN,2459241.5
K2-109,15333,HD-106315,Crossfield,Ian,2019-11-24,183.472466,-0.393417,WFC3 IR G141,437,NaN,NaN,NaN,NaN,2458811.5
WASP-178,16450,HD-134004,Lothringer,Joshua,2021-03-05,227.270309,-42.704976,WFC3 IR G141,152,NaN,NaN,NaN,NaN,2459278.5
LF 2 +31 176,16307,HD-185603B,Fu,Guangwei,2021-07-20,294.661419,31.219190,WFC3 IR G141,106,NaN,NaN,NaN,NaN,2459415.5
HD 195689,15820,HD-195689,Pino,Lorenzo,2020-01-25,307.859927,39.938944,WFC3 IR G141,216,NaN,NaN,NaN,NaN,2458873.5
HD 219666,15698,HD-219666,Beatty,Thomas,2019-11-12,349.557484,-56.903921,WFC3 IR G141,221,NaN,NaN,NaN,NaN,2458799.5


In [272]:
g

name                XO-2
pi_l            Griffith
pi_f             Caitlin
date          2016-01-06
ra            117.028824
dec            50.221054
mode        WFC3 IR G141
nfiles                87
title                NaN
bibcode              NaN
altnames             NaN
simbad               NaN
jd             2457393.5
Name: (XO-2, 13653), dtype: object

In [281]:
for idx, g in grp.iterrows():
    l = len(glob(f'/Volumes/Nibelheim/hst/data/{idx[0]}')) + len(glob(f'/Volumes/Nibelheim/hst/data/{g["name"]}'))
    if l == 0:
        print(idx[0], g['name'])

TRAPPIST-1 2MASS-J23062928-0502285
GJ   324 A 55-CANCRI
K2-135 BD-02-5958
CoRoT 101206560 COROT-2
LS   V +52   21 G191B2B
GJ  1214 GJ-1214
GJ  3470 GJ-3470
GJ  3470 GJ-3470
GJ   436 GJ-436
HAT-P-13 HAT-13
TIC 346338553 HAT-1C
HAT-P-2 HAT-P-2
K2-109 HD-106315
GJ  4130 HD-189733
HD 209458 HD-209458
K2-96 HD-3167
HD  93396 HD-93396
GJ  3651 HD-97658
GJ  3651 HD-97658
Kepler-51 KEPLER-51
L   98-59 L-98-59
GJ  1132 LHS-281
TIC 116264089 TRES-3
WASP-19 WASP-19


In [ ]:
BD-02-5958

In [283]:
import ombre as om

In [ ]:
om.Ob

In [260]:
grp[grp.name == 'LTT-1445A']

,,name,pi_l,pi_f,date,ra,dec,mode,nfiles,title,bibcode,altnames,simbad,jd
simbad,propid,,,,,,,,,,,,,
GJ 3193,16039,LTT-1445A,Sing,David,2021-01-18,45.463535,-16.59375,WFC3 IR G141,350,NaN,NaN,NaN,NaN,2459232.5


In [190]:
grp.to_csv('ch_literature_table.csv')

In [195]:
from astropy.time import Time

In [201]:
np.asarray(grp['date'])[0]

'2018-12-28'

[2458480.5,
 2457190.5,
 2457237.5,
 2458608.5,
 2457763.5,
 2457369.5,
 2457491.5,
 2458856.5,
 2459377.5,
 2458748.5,
 2455827.5,
 2457574.5,
 2457527.5,
 2455417.5,
 2456524.5,
 2458102.5,
 2456330.5,
 2457317.5,
 2456294.5,
 2457577.5,
 2455447.5,
 2456113.5,
 2455760.5,
 2456277.5,
 2457748.5,
 2457631.5,
 2456620.5,
 2457325.5,
 2457766.5,
 2459204.5,
 2457616.5,
 2457510.5,
 2457567.5,
 2457408.5,
 2457740.5,
 2457626.5,
 2457677.5,
 2457757.5,
 2459241.5,
 2457467.5,
 2458811.5,
 2459278.5,
 2457481.5,
 2458059.5,
 2456666.5,
 2456904.5,
 2459415.5,
 2455510.5,
 2458873.5,
 2457022.5,
 2458799.5,
 2459089.5,
 2458737.5,
 2458226.5,
 2456665.5,
 2457784.5,
 2455814.5,
 2456467.5,
 2456195.5,
 2458989.5,
 2458252.5,
 2458162.5,
 2457715.5,
 2458044.5,
 2457312.5,
 2457778.5,
 2458429.5,
 2459269.5,
 2458083.5,
 2459232.5,
 2459056.5,
 2455622.5,
 2457910.5,
 2457904.5,
 2457663.5,
 2455479.5,
 2455667.5,
 2456720.5,
 2457424.5,
 2457703.5,
 2458519.5,
 2458096.5,
 2458217.5,
 245

In [191]:
grp.tail(50)

name            pi_l     pi_f        date  \
simbad        propid                                                         
GJ  3651      13665          HD-97658         Benneke    Bjorn  2017-01-31   
GJ  4130      12181          HD189733          Deming    Drake  2011-09-10   
              12881          HD189733      McCullough    Peter  2013-06-24   
HD 209458     12181          HD209458          Deming    Drake  2012-09-25   
K2-93         15333         HIP-41378      Crossfield      Ian  2020-05-20   
K2-18         14682             K2-18         Benneke    Bjorn  2018-05-14   
K2-3          14682              K2-3         Benneke    Bjorn  2018-02-13   
TOI-1476      14664            KELT-1          Beatty   Thomas  2016-11-23   
HD  33643     14767            KELT-7            Sing    David  2017-10-18   
Kepler-138    13665        KEPLER-138         Benneke    Bjorn  2015-10-17   
Kepler-51     14218         KEPLER-51  Berta-Thompson     Zach  2017-01-25   
Kepler-79     15138         KEPLER-79   Jontof-Hutter   Daniel  2018-11-07   
L   98-59     15856           L-98-59         Barclay   Thomas  2021-02-24   
GJ  1132      14758           LHS-281  Berta-Thompson     Zach  2017-11-26   
GJ  3193      16039         LTT-1445A            Sing    David  2021-01-18   
TOI-674       15333            TOI674      Crossfield      Ian  2020-07-26   
TIC 116264089 12181            TRES-3          Deming    Drake  2011-03-02   
WASP-107      14915   TYC-5530-1795-1       Kreidberg    Laura  2017-06-06   
              14916   TYC-5530-1795-1           Spake  Jessica  2017-05-31   
WASP-101      14767          WASP-101            Sing    David  2016-10-02   
WASP-12       12181           WASP-12          Deming    Drake  2010-10-10   
              12230           WASP-12           Swain     Mark  2011-04-16   
              13467           WASP-12            Bean    Jacob  2014-03-04   
WASP-121      14468          WASP-121     Mikal-Evans   Thomas  2016-02-06   
              14767          WASP-121            Sing    David  2016-11-11   
              15134          WASP-121     Mikal-Evans   Thomas  2019-02-05   
              15135          WASP-121     Mikal-Evans   Thomas  2017-12-09   
WASP-127      14619          WASP-127           Spake  Jessica  2018-04-09   
WASP-17       12181           WASP-17          Deming    Drake  2011-07-08   
              14918           WASP-17        Wakeford   Hannah  2017-09-25   
WASP-18       12181           WASP-18          Deming    Drake  2011-06-25   
              13467           WASP-18            Bean    Jacob  2014-08-27   
WASP-19       12181           WASP-19          Deming    Drake  2011-07-01   
WASP-29       14260           WASP-29          Deming    Drake  2016-04-15   
WASP-31       12473           WASP-31            Sing    David  2012-05-13   
WASP-33       12495           WASP-33          Deming    Drake  2013-01-14   
WASP-39       14260           WASP-39          Deming    Drake  2017-02-07   
WASP-43       13467           WASP-43            Bean    Jacob  2013-12-05   
WASP-52       14260           WASP-52          Deming    Drake  2016-08-28   
WASP-6        14767            WASP-6            Sing    David  2017-05-06   
WASP-63       14642           WASP-63       Stevenson    Kevin  2016-09-19   
WASP-67       14260           WASP-67          Deming    Drake  2016-10-22   
WASP-69       14260           WASP-69          Deming    Drake  2016-08-16   
WASP-74       14767           WASP-74            Sing    David  2017-05-02   
WASP-76       14260           WASP-76          Deming    Drake  2015-11-26   
              14767           WASP-76            Sing    David  2016-11-03   
WASP-79       14767           WASP-79            Sing    David  2017-03-03   
WASP-80       14260           WASP-80          Deming    Drake  2016-06-21   
XO-1          12181              XO-1          Deming    Drake  2011-09-30   
XO-2          13653              XO-2        Griffith  

In [85]:
tab

name        pi_l     pi_f        date  \
simbad     propid                                                             
WASP-96    15469   2MASS-J00041112-4721382     Nikolov  Nikolay  2018-12-28   
WASP-103   13660   2MASS-J16371556+0711000        Zhao     Ming  2015-06-17   
           14050   2MASS-J16371556+0711000   Kreidberg    Laura  2015-08-03   
NaN        15685   2MASS-J19432996+5059289  Vanderburg   Andrew  2019-05-05   
TRAPPIST-1 14873   2MASS-J23062928-0502285      de Wit   Julien  2017-01-10   
...                                    ...         ...      ...         ...   
WASP-76    14767                   WASP-76        Sing    David  2016-11-03   
WASP-79    14767                   WASP-79        Sing    David  2017-03-03   
WASP-80    14260                   WASP-80      Deming    Drake  2016-06-21   
XO-1       12181                      XO-1      Deming    Drake  2011-09-30   
NaN        13653                      XO-2    Griffith  Caitlin  2016-01-06   

                           ra        dec          mode  nfiles  \
simbad     propid                                                
WASP-96    15469     1.046530 -47.360605  WFC3 IR G141     170   
WASP-103   13660   249.314867   7.183353  WFC3 IR G141     238   
           14050   249.314867   7.183353  WFC3 IR G141     536   
NaN        15685   295.874869  50.991414  WFC3 IR G141     344   
TRAPPIST-1 14873   346.626542  -5.043583  WFC3 IR G141     360   
...                       ...        ...           ...     ...   
WASP-76    14767    26.632967   2.700393  WFC3 IR G141      94   
WASP-79    14767    66.370918 -30.600416  WFC3 IR G141     130   
WASP-80    14260   303.167500  -2.144222  WFC3 IR G141      75   
XO-1       12181   240.549260  28.169604  WFC3 IR G141     129   
NaN        13653   117.028824  50.221054  WFC3 IR G141      87   

                                                               title  \
simbad     propid                                                      
WASP-96    15469   [climate of an ultra hot jupiter. spectroscopi...   
WASP-103   13660   [high-precision multiwavelength eclipse photom...   
           14050   [high-precision multiwavelength eclipse photom...   
NaN        15685                                                 NaN   
TRAPPIST-1 14873   [the near-infrared transmission spectra of tra...   
...                                                              ...   
WASP-76    14767   [ares i: wasp-76 b, a tale of two hst spectra,...   
WASP-79    14767   [climate of an ultra hot jupiter. spectroscopi...   
WASP-80    14260   [the impact of non-uniform thermal structure o...   
XO-1       12181   [infrared transmission spectroscopy of the exo...   
NaN        13653                                                 NaN   

                                                             bibcode  \
simbad     propid                                                      
WASP-96    15469          [2019A&A...625A.136A, 2021AJ....161....4Y]   
WASP-103   13660   [2018MNRAS.474.2334D, 2019MNRAS.489..941P, 202...   
           14050   [2018MNRAS.474.2334D, 2019MNRAS.489..941P, 202...   
NaN        15685                                                 NaN   
TRAPPIST-1 14873   [2018AJ....156..178Z, 2018AJ....156..218D, 202...   
...                                                              ...   
WASP-76    14767          [2020AJ....160....8E, 2018AJ....155..156T]   
WASP-79    14767   [2019A&A...625A.136A, 2020AJ....159....5S, 202...   
WASP-80    14260                               [2016ApJ...829...52F]   
XO-1       12181          [2013ApJ...774...95D, 2018MNRAS.481.4261S]   
NaN        13653                                                 NaN   

                                                            altnames  
simbad     propid                                                     
WASP-96    15469   [1SWASP J000411.14-472138.2, 2MASS J00041112-4...  
WASP-103   13660   [1SWASP J163715.59+071100.0, 2MASS J16371556+

In [71]:
tab.to_csv('ch_literature_table.csv')

In [72]:
tab.simbad

name         propid
G191B2B      11926     LS   V +52   21
HAT-13       12181                 NaN
WASP-12      12181             WASP-12
HD-189733    11740                 NaN
TRES-3       12181                 NaN
                            ...       
HAT-P-70B    16307                 NaN
L-98-59      15856                 NaN
HD-134004    16450                 NaN
CD-38-15670  16457                 NaN
HD-185603B   16307                 NaN
Name: simbad, Length: 105, dtype: object

In [55]:
tab.head(40)

,,pi_l,pi_f,date,ra,dec,mode,nfiles,title,bibcode,altnames,simbad
name,propid,,,,,,,,,,,
G191B2B,11926,Deustua,Susana,2010-08-09,76.377550,52.831094,WFC3 IR G141,26,[ultralytics/yolov5: v5.0 - yolov5-p6 1280 mod...,"[2021zndo...3908559J, 2021arXiv210801382V, 201...","[** WNO 44B, 1RXS J050525.8+524913, 1SWASP J...",LS V +52 21
HAT-13,12181,Deming,Drake,2010-09-08,129.882390,47.351946,WFC3 IR G141,200,NaN,NaN,NaN,NaN
WASP-12,12181,Deming,Drake,2010-10-10,97.636667,29.672222,WFC3 IR G141,28,[exoplanet transit spectroscopy using wfc3: wa...,"[2013ApJ...779..128M, 2015ApJ...801...86K, 201...","[2MASS J06303279+2940202, Gaia DR1 34352828624...",WASP-12
HD-189733,11740,Pont,Frederic,2010-11-10,300.182120,22.710114,WFC3 IR G141,216,NaN,NaN,NaN,NaN
TRES-3,12181,Deming,Drake,2011-03-02,268.029079,37.546266,WFC3 IR G141,220,NaN,NaN,NaN,NaN
WASP-12,12230,Swain,Mark,2011-04-16,97.636639,29.672278,WFC3 IR G141,1160,[exoplanet transit spectroscopy using wfc3: wa...,"[2013ApJ...779..128M, 2015ApJ...801...86K, 201...","[2MASS J06303279+2940202, Gaia DR1 34352828624...",WASP-12
WASP-18,12181,Deming,Drake,2011-06-25,24.354295,-45.677887,WFC3 IR G141,312,[estimating dayside effective temperatures of ...,"[2019MNRAS.489..941P, 2019A&A...625A.136A, 202...","[2MASS J01372503-4540404, CD-46 449, CPC 0 ...",WASP-18
WASP-19,12181,Deming,Drake,2011-07-01,148.416812,-45.659142,WFC3 IR G141,550,[exoplanet transit spectroscopy using wfc3: wa...,"[2013ApJ...779..128M, 2013MNRAS.434.3252H, 201...","[2MASS J09534008-4539330, ASTEP A-009-1934-226...",WASP-19
WASP-17,12181,Deming,Drake,2011-07-08,239.962263,-28.061785,WFC3 IR G141,132,[exoplanet transit spectroscopy using wfc3: wa...,"[2013ApJ...779..128M, 2021arXiv210813721S, 201...","[1SWASP J155950.94-280342.3, 2MASS J15595095-2...",WASP-17
